In [5]:
# some notes from SAE tutorial on gemmascope https://colab.research.google.com/drive/17dQFYUYnuKnP6OwQPH9v_GSYUW5aj-Rp#scrollTo=12wF3f7o1Ni7
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import numpy as np

# this is what you do if you only want inference (not training)
# saves on memory usage
torch.set_grad_enabled(False) 

/Users/enjalot/code/touch-tokens/ttenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# every transformer has a tokenizer, so we load the one for the model we want to use
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b")
# this downloads the model (or loads it from disk cache)
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b")

Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00, 15.88it/s]


In [7]:
# we want to study what's happening in the model when we run some input text through it
input_text = "hello, Yoda my name is"
# the first step is to tokenize the input text
input_ids = tokenizer(input_text, return_tensors="pt", add_special_tokens=True)

In [8]:
input_ids['input_ids']

tensor([[     2,  17534, 235269, 146433,    970,   1503,    603]])

In [9]:
for t in input_ids['input_ids'][0]:
  print(f"Token {t}: \t\t{tokenizer.decode(t)}")

Token 2: 		<bos>
Token 17534: 		hello
Token 235269: 		,
Token 146433: 		 Yoda
Token 970: 		 my
Token 1503: 		 name
Token 603: 		 is


In [10]:
# now we can run our model. Let's generate just 2 more tokens for now
outputs = model.generate(**input_ids, max_new_tokens=2)
print("output tokens", len(outputs[0]))
outputs[0]


The 'max_batch_size' argument of HybridCache is deprecated and will be removed in v4.46. Use the more precisely named 'batch_size' argument instead.


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


output tokens 9


tensor([     2,  17534, 235269, 146433,    970,   1503,    603, 146433,    578])

In [11]:
# we need to turn the output tokens back into text (the output includes the input)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
generated_text

'hello, Yoda my name is Yoda and'

In [12]:
# To get hidden states for the input, we need to run the model again with the generated sequence
hidden_output = model(**input_ids, output_hidden_states=True)

# Access hidden states
hidden_states = hidden_output.hidden_states


In [13]:

print("input tokens", len(input_ids['input_ids'][0]))
print("hidden states (layers)", len(hidden_states))
print("hidden state shape", hidden_states[20].shape)

input tokens 7
hidden states (layers) 27
hidden state shape torch.Size([1, 7, 2304])


In [14]:
print(model)

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (pre_feedforward_layernorm): Gemma2RMSNorm((2304,), eps

In [15]:
print(f"Number of layers in config: {model.config.num_hidden_layers}")
print(f"Hidden size: {model.config.hidden_size}")

Number of layers in config: 26
Hidden size: 2304


In [16]:
def gather_residual_activations(model, target_layer, inputs):
  """
  This function allows us to gather activations for a specific layer on a model.
  
  Args:
  - model: The model from which we want to gather activations.
  - target_layer: The specific layer index for which we want to gather activations.
  - inputs: The input data to be passed through the model.
  
  Returns:
  - target_act: The activations of the specified layer.
  """
  target_act = None
  def gather_target_act_hook(mod, inputs, outputs):
    nonlocal target_act # make sure we can modify the target_act from the outer scope
    target_act = outputs[0]
    return outputs
  # we could also easily target the MLP layer
  # handle = model.model.layers[target_layer].mlp.register_forward_hook(gather_mlp_output_hook)
  handle = model.model.layers[target_layer].register_forward_hook(gather_target_act_hook)
  _ = model.forward(inputs)
  handle.remove()
  return target_act

In [17]:
# the 20th index is actually the 21st layer
target_act = gather_residual_activations(model, 20, input_ids['input_ids'])

In [18]:
target_act.shape

torch.Size([1, 7, 2304])

In [19]:
target_act

tensor([[[ 1.9440,  1.7632, -2.0879,  ...,  1.6978, -2.0868, -0.0178],
         [-7.1130,  2.3935, -0.3611,  ..., -0.0207,  4.9698,  2.0012],
         [-8.1484,  0.2589, -0.5944,  ..., -1.2177,  2.7641,  2.0122],
         ...,
         [ 1.5289, -3.7196,  7.7939,  ..., -9.8385,  0.1159,  0.6954],
         [-3.1761,  1.1008,  0.5456,  ..., -1.7524, -2.0917,  2.4893],
         [ 0.9512, -2.4978, -0.4893,  ..., -4.7208, -5.6637, -0.7142]]])

In [20]:
hidden_states[21]

tensor([[[ 1.9440,  1.7632, -2.0879,  ...,  1.6978, -2.0868, -0.0178],
         [-7.1130,  2.3935, -0.3611,  ..., -0.0207,  4.9698,  2.0012],
         [-8.1484,  0.2589, -0.5944,  ..., -1.2177,  2.7641,  2.0122],
         ...,
         [ 1.5289, -3.7196,  7.7939,  ..., -9.8385,  0.1159,  0.6954],
         [-3.1761,  1.1008,  0.5456,  ..., -1.7524, -2.0917,  2.4893],
         [ 0.9512, -2.4978, -0.4893,  ..., -4.7208, -5.6637, -0.7142]]])

## SAE Time

In [21]:
from huggingface_hub import hf_hub_download

In [22]:
# We download the weights for the SAE we want to use
# https://www.neuronpedia.org/gemma-2-2b/20-gemmascope-res-16k
path_to_params = hf_hub_download(
    repo_id="google/gemma-scope-2b-pt-res",
    filename="layer_20/width_16k/average_l0_71/params.npz",
    force_download=False,
)
params = np.load(path_to_params)
pt_params = {k: torch.from_numpy(v) for k, v in params.items()}

In [23]:
import torch.nn as nn
class JumpReLUSAE(nn.Module):
  def __init__(self, d_model, d_sae):
    # Note that we initialise these to zeros because we're loading in pre-trained weights.
    # If you want to train your own SAEs then we recommend using blah
    super().__init__()
    self.W_enc = nn.Parameter(torch.zeros(d_model, d_sae))
    self.W_dec = nn.Parameter(torch.zeros(d_sae, d_model))
    self.threshold = nn.Parameter(torch.zeros(d_sae))
    self.b_enc = nn.Parameter(torch.zeros(d_sae))
    self.b_dec = nn.Parameter(torch.zeros(d_model))

  def encode(self, input_acts):
    pre_acts = input_acts @ self.W_enc + self.b_enc
    mask = (pre_acts > self.threshold)
    acts = mask * torch.nn.functional.relu(pre_acts)
    return acts

  def decode(self, acts):
    return acts @ self.W_dec + self.b_dec

  def forward(self, acts):
    acts = self.encode(acts)
    recon = self.decode(acts)
    return recon

In [24]:
params['W_enc'].shape[0]

2304

In [25]:
sae = JumpReLUSAE(params['W_enc'].shape[0], params['W_enc'].shape[1])
sae.load_state_dict(pt_params)

<All keys matched successfully>

In [26]:
sae_acts = sae.encode(target_act.to(torch.float32))
reconstruction = sae.decode(sae_acts)

In [27]:
sae_acts.shape

torch.Size([1, 7, 16384])

In [28]:
reconstruction.shape

torch.Size([1, 7, 2304])

In [29]:
values, inds = sae_acts.max(-1)

inds

tensor([[ 6631, 14956, 10299, 15449, 11302,  8564, 15449]])

In [30]:
values

tensor([[2028.7983,  122.3900,  107.6448,   90.2571,   89.6321,  102.8196,
           42.7972]])

In [31]:
k = 5  # Change this to the number of top values you want
values, indices = torch.topk(sae_acts, k, dim=-1)

print(f"Top {k} values shape: {values.shape}")
print(f"Top {k} indices shape: {indices.shape}")

# Print the top k values and indices for the first sequence item
print(f"\nTop {k} values for first sequence item:")
print(values[0, 0])
print(f"\nTop {k} indices for first sequence item:")
print(indices[0, 0])

Top 5 values shape: torch.Size([1, 7, 5])
Top 5 indices shape: torch.Size([1, 7, 5])

Top 5 values for first sequence item:
tensor([2028.7983,  781.3959,  534.8594,  264.1917,  252.5279])

Top 5 indices for first sequence item:
tensor([ 6631,   743,  5052, 16057,  9479])


In [32]:
from IPython.display import IFrame
html_template = "https://neuronpedia.org/{}/{}/{}?embed=true&embedexplanation=true&embedplots=true&embedtest=true&height=300"

def get_dashboard_html(sae_release = "gemma-2-2b", sae_id="20-gemmascope-res-16k", feature_idx=0):
    return html_template.format(sae_release, sae_id, feature_idx)



In [38]:
input_text

'hello, Yoda my name is'

In [46]:
for i,t in enumerate(input_ids['input_ids'][0]):
  print(f"{i} Token {t}: \t\t{tokenizer.decode(t)}\t\t{indices[0][i]}")

0 Token 2: 		<bos>		tensor([ 6631,   743,  5052, 16057,  9479])
1 Token 17534: 		hello		tensor([14956, 11527,  6779,  8684, 11795])
2 Token 235269: 		,		tensor([10299, 10881, 14956, 10736,  6779])
3 Token 146433: 		 Yoda		tensor([15449,  6631,  3703, 13851,  9768])
4 Token 970: 		 my		tensor([11302, 14239,  3800, 15449,  9768])
5 Token 1503: 		 name		tensor([ 8564,  2699,  9768,  3567, 11996])
6 Token 603: 		 is		tensor([15449,  9768, 11339,  4418, 13414])


In [50]:
html = get_dashboard_html(sae_release = "gemma-2-2b", sae_id="20-gemmascope-res-16k", feature_idx=inds[0][3].item())
IFrame(html, width=1200, height=600)

In [51]:

def modify_layer_activation(model, target_layer, input_ids, sae, feature_index, modification_value, max_new_tokens):
    """
    Modify the activation of a specific feature in a given layer.
    
    Args:
    - model: The LLM model
    - target_layer: The index of the layer to modify
    - input_ids: The input token IDs
    - sae: The Sparse Autoencoder
    - feature_index: The index of the feature to modify
    - modification_value: The value to add to the feature's activation
    
    Returns:
    - modified_output: The model's output after modification
    """
    def capture_and_modify_hook(module, inputs, outputs):
        # Capture the original activation
        original_act = outputs[0].detach()
        
        # Encode the activations using the SAE
        sae_acts = sae.encode(original_act.to(torch.float32))
        
        # Modify the specific feature's activation
        sae_acts[0, :, feature_index] += modification_value
        
        # Decode the modified activations
        modified_act = sae.decode(sae_acts)
        
        # Return the modified activation
        return (modified_act,) + outputs[1:]

    # Register the hook
    handle = model.model.layers[target_layer].register_forward_hook(capture_and_modify_hook)
    
    # Run the model with the modified activation
    with torch.no_grad():
        modified_output = model.generate(input_ids, max_new_tokens=max_new_tokens)
    
    # Remove the hook
    handle.remove()
    
    return modified_output


In [52]:
# we want to study what's happening in the model when we run some input text through it
input_text2 = "Hello, my name is"
# the first step is to tokenize the input text
input_ids2 = tokenizer(input_text2, return_tensors="pt", add_special_tokens=True)
max_new_tokens = 10

In [53]:
outputs2 = model.generate(**input_ids2, max_new_tokens=max_new_tokens)
generated_text2 = tokenizer.decode(outputs2[0], skip_special_tokens=True)

KeyboardInterrupt: 

In [37]:
target_layer = 20  # The layer you want to modify
feature_index = 15449  # The feature index you want to modify
modification_value = 1000.0  # The value to add to the feature's activation

modified_output = modify_layer_activation(model, target_layer, input_ids2['input_ids'], sae, feature_index, modification_value, max_new_tokens)

# Generate text from the modified output
modified_text = tokenizer.decode(modified_output[0], skip_special_tokens=True)

print("Original text:", generated_text2)
print("Modified text:", modified_text)


KeyboardInterrupt: 